# Data Preprocessing for Visualization in Tableau

In [20]:
import json
import csv
import re

Meetup Data from Json to Csv

In [4]:
# Open the JSON file
with open('meetup-20240407.json') as file:
    data = json.load(file)

# Define the CSV file where the output must be stored
csv_file = open('event.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

# Assuming that data is a list of dictionaries
count = 0
for item in data:
    if count == 0:
        # Writing headers of CSV file
        header = item.keys()
        csv_writer.writerow(header)
        count += 1
    # Check if '_id' is in the dictionary and format it
    if '_id' in item and '$oid' in item['_id']:
        item['_id'] = item['_id']['$oid']
    # Writing data of CSV file
    csv_writer.writerow(item.values())

csv_file.close()

Posts from Json to Csv

In [5]:
with open('post-20240407.json') as file:
    data = json.load(file)

csv_file = open('post.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

count = 0
for item in data:
    if count == 0:
        header = item.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(item.values())

csv_file.close()

second post data 

In [7]:
with open('system_data.postd_ata.json') as file:
    data = json.load(file)

csv_file = open('1post.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

count = 0
for item in data:
    if count == 0:
        header = item.keys()
        csv_writer.writerow(header)
        count += 1
    if '_id' in item and '$oid' in item['_id']:
        item['_id'] = item['_id']['$oid']
    csv_writer.writerow(item.values())

csv_file.close()

In [ ]:
user data

In [8]:
with open('system_data.user_info.json') as file:
    data = json.load(file)

csv_file = open('user-data.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

count = 0
for item in data:
    if count == 0:
        header = item.keys()
        csv_writer.writerow(header)
        count += 1
    if '_id' in item and '$oid' in item['_id']:
        item['_id'] = item['_id']['$oid']
    csv_writer.writerow(item.values())

csv_file.close()

In [ ]:
gym data

In [15]:
def convert_txt_to_csv('gym-data.txt', 'gym-data1.csv', delimiter=','):

    df = pd.read_csv('gym-data.txt', delimiter=delimiter)
    df.to_csv('gym-data1.csv', index=False)

convert_txt_to_csv('gym-data.txt', 'gym-data1.csv')

SyntaxError: invalid syntax (<ipython-input-15-bdeecce33b74>, line 1)

In [23]:
import re
import html

In [30]:
def clean_json_string(raw_json):
    """
    Decodes HTML entities and attempts to correct common JSON formatting issues.
    """
    # Decode HTML entities in the JSON string
    decoded_json = html.unescape(raw_json)
    # Remove embedded HTML tags and fix links
    decoded_json = re.sub(r'\\u003c[a-zA-Z\/][^>]*\\u003e', '', decoded_json)
    # Standard JSON clean-up steps
    decoded_json = decoded_json.replace('\\"', '"')
    decoded_json = decoded_json.replace('\\n', '')
    decoded_json = decoded_json.replace('\\\\', '\\')
    decoded_json = re.sub(r'\\', '', decoded_json)
    decoded_json = re.sub(r'\}\s*\{', '},{', decoded_json)
    decoded_json = re.sub(r'\s*,\s*}', '}', decoded_json)
    decoded_json = re.sub(r'\s*,\s*\]', ']', decoded_json)
    return decoded_json

def extract_json_data(filepath):
    """
    Extracts and cleans JSON data from a file, managing non-JSON headers and complex structures.
    """
    try:
        with open(filepath, 'r') as file:
            content = file.read()
        # Find the start of the actual JSON object
        json_start_pos = content.find('{')
        if json_start_pos == -1:
            print("No JSON object found in the file.")
            return None
        # Extract and clean the JSON part
        json_content = content[json_start_pos:]
        clean_content = clean_json_string(json_content)
        # Try parsing the cleaned content
        return json.loads(clean_content)
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        error_position = e.pos
        print("Error context (100 characters around error):", json_content[max(0, error_position-50):error_position+50])
        return None
    except Exception as e:
        print(f"Error reading or parsing file: {e}")
        return None

def convert_json_to_csv(json_data, output_csv_path):
    """
    Converts JSON data to CSV.
    """
    try:
        # Handle both single dict and list of dicts
        if isinstance(json_data, dict):
            json_data = [json_data]
        df = pd.json_normalize(json_data)
        df.to_csv(output_csv_path, index=False)
        print(f"Data successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")

# Usage
json_data = extract_json_data('gym-data.txt')  # Replace with your actual file path
if json_data:
    convert_json_to_csv(json_data, 'output.csv')

JSON Decode Error: Extra data: line 73 column 2 (char 2558)
Error context (100 characters around error):       "user_ratings_total" : 93
      },
      {
         "business_status" : "OPERATIONAL",
       


In [33]:
def clean_json_string(raw_json):
    """
    Performs comprehensive cleaning of the JSON string, addressing multiple potential issues.
    """
    # Decode HTML entities and remove embedded HTML-like content
    decoded_json = html.unescape(raw_json)
    decoded_json = re.sub(r'\\u003c[a-zA-Z\/][^>]*\\u003e', '', decoded_json)
    
    # Fix common escape sequence issues and backslash problems
    decoded_json = decoded_json.replace('\\"', '"')
    decoded_json = decoded_json.replace('\\n', '')
    decoded_json = decoded_json.replace('\\\\', '\\')
    decoded_json = re.sub(r'\\', '', decoded_json)

    # Correct multiple JSON objects and add commas if needed between them
    decoded_json = re.sub(r'(?<=\})\s*(?=\{)', '},{', decoded_json)

    # Handle trailing non-JSON content by ending at the last valid JSON character
    last_brace_pos = decoded_json.rfind('}')
    if last_brace_pos != -1:
        decoded_json = decoded_json[:last_brace_pos + 1]

    # Ensure the entire content is treated as an array of objects
    return '[' + decoded_json + ']'

def extract_json_data(filepath):
    """
    Extracts and cleans JSON data from a file, ensuring valid format and structure.
    """
    try:
        with open(filepath, 'r') as file:
            content = file.read()

        # Find the start of actual JSON data (first '{') and remove non-JSON headers
        json_start_pos = content.find('{')
        if json_start_pos == -1:
            print("No JSON object found in the file.")
            return None
        
        json_content = content[json_start_pos:]
        clean_content = clean_json_string(json_content)
        return json.loads(clean_content)
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        error_position = e.pos
        print("Error context (100 characters around error):", clean_content[max(0, error_position-50):error_position+50])
        return None
    except Exception as e:
        print(f"Error reading or parsing file: {e}")
        return None

def convert_json_to_csv(json_data, output_csv_path):
    """
    Converts JSON data to a CSV file.
    """
    try:
        df = pd.json_normalize(json_data)
        df.to_csv(output_csv_path, index=False)
        print(f"Data successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")

# Usage
json_data = extract_json_data('gym-data.txt')  # Update with your actual file path
if json_data:
    convert_json_to_csv(json_data, 'output.csv')

Data successfully saved to output.csv


In [34]:
def clean_json_string(raw_json):
    """
    Cleans JSON string by decoding HTML entities, removing HTML-like content,
    and ensuring proper formatting for multiple JSON objects.
    """
    decoded_json = html.unescape(raw_json)
    decoded_json = re.sub(r'\\u003c[a-zA-Z\/][^>]*\\u003e', '', decoded_json)
    decoded_json = decoded_json.replace('\\"', '"')
    decoded_json = decoded_json.replace('\\n', '')
    decoded_json = decoded_json.replace('\\\\', '\\')
    decoded_json = re.sub(r'\\', '', decoded_json)
    decoded_json = re.sub(r'(?<=\})\s*(?=\{)', '},{', decoded_json)
    last_brace_pos = decoded_json.rfind('}')
    if last_brace_pos != -1:
        decoded_json = decoded_json[:last_brace_pos + 1]
    return '[' + decoded_json + ']'

def extract_json_data(filepath):
    """
    Extracts and cleans JSON data from a file, ensuring valid format and structure.
    """
    try:
        with open(filepath, 'r') as file:
            content = file.read()
        json_start_pos = content.find('{')
        if json_start_pos == -1:
            print("No JSON object found in the file.")
            return None
        json_content = content[json_start_pos:]
        clean_content = clean_json_string(json_content)
        json_data = json.loads(clean_content)
        print("JSON Data Preview:", json.dumps(json_data, indent=4, sort_keys=True)[:500])  # Preview the first 500 characters of the JSON data
        return json_data
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        error_position = e.pos
        print("Error context (100 characters around error):", clean_content[max(0, error_position-50):error_position+50])
        return None
    except Exception as e:
        print(f"Error reading or parsing file: {e}")
        return None

def convert_json_to_csv(json_data, output_csv_path):
    """
    Converts JSON data to a CSV file.
    """
    try:
        df = pd.json_normalize(json_data)
        print("DataFrame Preview:", df.head())  # Preview the first few rows of the DataFrame
        df.to_csv(output_csv_path, index=False)
        print(f"Data successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")

# Usage
json_data = extract_json_data('gym-data.txt')  # Replace with your actual file path
if json_data:
    convert_json_to_csv(json_data, 'output.csv')

JSON Data Preview: [
    {
        "html_attributions": [],
        "next_page_token": "ATplDJYU7B4BzbYcu8_u-2yZD9NiT_jVyKjx_AkARe0dJUj27cCTjIk0ei9OX095kw3hd5tbjJiKZsWzJZgBP_nN4sgq4A8qv58B2gURy59d409nBDF6qjud9PvZkLmYdFZdzGHIgMPIS3xvz3bTseg7HTdTelBBoFkmp9Mo4ciLE9QPkEEYxtmsFtywheoO4ZY_Av4s1oDpf2z7f8ZT1osIdt7UvhH7tlBGeiSbIQ-6YCAxZv7sX6Sk8qxdykmrz8yZW2lL4lztJ3RLiaZKWavYnbo1VuY1hvaBwJPW3mYx9KCltVuKk_BK6p6pnkU1eJ8bZe5ykqlw4xrNhxoSAFv9-nbEyNLwLLsnsQlz3P6HWyxHxOJP1_wA4Weok3yuK49MbyhM_jWnXP32c49St3o",
        "results": [

DataFrame Preview:   html_attributions                                    next_page_token  \
0                []  ATplDJYU7B4BzbYcu8_u-2yZD9NiT_jVyKjx_AkARe0dJU...   

                                             results status  
0  [{'business_status': 'OPERATIONAL', 'formatted...     OK  
Data successfully saved to output.csv


In [36]:
def convert_json_to_csv(input_file_path, output_file_path):
    # Reading the entire text file to extract the JSON data
    with open(input_file_path, 'r') as file:
        full_content = file.read()

    # Extracting the JSON part of the text
    json_start_index = full_content.find('{')
    json_end_index = full_content.rfind('}') + 1  # +1 to include the last curly brace
    json_data = full_content[json_start_index:json_end_index]

    # Converting the JSON string into a dictionary
    json_dict = json.loads(json_data)
    results = json_dict.get('results', [])  # 'results' should contain the list of gyms

    # Fields to be included in the CSV file
    fields = ['name', 'business_status', 'formatted_address', 'latitude', 'longitude', 'rating', 'user_ratings_total']

    # Writing to CSV
    with open(output_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fields)
        writer.writeheader()
        for gym in results:
            row = {
                'name': gym.get('name', 'N/A'),
                'business_status': gym.get('business_status', 'N/A'),
                'formatted_address': gym.get('formatted_address', 'N/A'),
                'latitude': gym.get('geometry', {}).get('location', {}).get('lat', 'N/A'),
                'longitude': gym.get('geometry', {}).get('location', {}).get('lng', 'N/A'),
                'rating': gym.get('rating', 'N/A'),
                'user_ratings_total': gym.get('user_ratings_total', 'N/A')
            }
            writer.writerow(row)

# Example usage of the function
input_file_path = 'gym-data.txt'  # Change this to your actual file path
output_file_path = 'output_csv_file.csv'
convert_json_to_csv(input_file_path, output_file_path)

In [ ]:
museum data

In [41]:
def convert_json_to_csv(input_file_path, output_file_path):
    # Open the input file and read the data
    with open(input_file_path, 'r') as file:
        data = file.read()

    # Try to find the JSON object starting and ending points
    start_index = data.find('{')
    end_index = data.rfind('}') + 1  # Include the last curly brace
    
    # Extract the JSON substring
    json_data = data[start_index:end_index]

    # Attempt to parse the JSON data
    try:
        parsed_json = json.loads(json_data)
        results = parsed_json.get('results', [])
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
        return

    # Define the fields to be included in the CSV
    fields = [
        'name', 'business_status', 'formatted_address', 'latitude', 'longitude',
        'rating', 'user_ratings_total'
    ]

    # Write to the CSV file
    with open(output_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fields)
        writer.writeheader()
        for item in results:
            row = {
                'name': item['name'],
                'business_status': item['business_status'],
                'formatted_address': item['formatted_address'],
                'latitude': item['geometry']['location']['lat'],
                'longitude': item['geometry']['location']['lng'],
                'rating': item.get('rating', 'N/A'),
                'user_ratings_total': item.get('user_ratings_total', 'N/A')
            }
            writer.writerow(row)

# Example usage
input_file_path = 'museum-data.txt'
output_file_path = 'museum_data.csv'
convert_json_to_csv(input_file_path, output_file_path)

Failed to decode JSON: Extra data: line 61 column 8 (char 2053)


In [44]:
def clean_json_content(raw_json):
    # Replace improperly quoted keys and values
    corrected_json = re.sub(r"(?<=[:{\[,]\s*)'([^']+)'(?=\s*[:,\]})", r'"\1"', raw_json)
    # Ensure that numeric and boolean values do not get quotes
    corrected_json = re.sub(r'"\s*([0-9]+|true|false)\s*"', r'\1', corrected_json)
    # Fix unquoted keys
    corrected_json = re.sub(r'([{,])(\s*)(\w+)(\s*:\s*)', r'\1"\3"\4', corrected_json)
    return corrected_json

def convert_json_to_csv(input_file_path, output_file_path):
    # Read the JSON data from the file
    with open(input_file_path, 'r') as file:
        file_content = file.read()
    
    # Clean the JSON string to fix common formatting errors
    cleaned_json = clean_json_content(file_content)
    
    # Attempt to parse the corrected JSON structure
    try:
        json_data = json.loads(cleaned_json)
        museum_data = json_data['results']
    
        # Convert data to a DataFrame
        df_museums = pd.DataFrame({
            'Name': [museum['name'] for museum in museum_data],
            'Address': [museum['formatted_address'] for museum in museum_data],
            'Latitude': [museum['geometry']['location']['lat'] for museum in museum_data],
            'Longitude': [museum['geometry']['location']['lng'] for museum in museum_data],
            'Business Status': [museum['business_status'] for museum in museum_data],
            'Rating': [museum.get('rating', None) for museum in museum_data],
            'Total User Ratings': [museum.get('user_ratings_total', 0) for museum in museum_data]
        })

        # Save the DataFrame to a CSV file
        df_museums.to_csv(output_file_path, index=False)
        print(f"Data successfully written to {output_file_path}")
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")

# Usage
input_path = 'museum-data.txt'
output_path = 'museum.csv'
convert_json_to_csv(input_path, output_path)

error: unterminated character set at position 30

In [45]:
import json
import pandas as pd

def convert_json_to_csv(input_file_path, output_file_path):
    # Read the JSON data from the file
    with open(input_file_path, 'r') as file:
        file_content = file.read()

    # Parse the JSON data
    json_data = json.loads(file_content)

    # Extract data to a DataFrame
    restaurant_data = json_data['results']
    df_restaurants = pd.DataFrame({
        'Name': [restaurant['name'] for restaurant in restaurant_data],
        'Address': [restaurant['formatted_address'] for restaurant in restaurant_data],
        'Latitude': [restaurant['geometry']['location']['lat'] for restaurant in restaurant_data],
        'Longitude': [restaurant['geometry']['location']['lng'] for restaurant in restaurant_data],
        'Business Status': [restaurant['business_status'] for restaurant in restaurant_data],
        'Rating': [restaurant.get('rating', None) for restaurant in restaurant_data],
        'Total User Ratings': [restaurant.get('user_ratings_total', 0) for restaurant in restaurant_data]
    })

    # Save the DataFrame to a CSV file
    df_restaurants.to_csv(output_file_path, index=False)
    print(f"Data successfully written to {output_file_path}")

# Usage
input_path = 'restaurant-data.txt'  # Replace with the path to your JSON file
output_path = 'restaurant.csv'        # Replace with your desired output CSV file path
convert_json_to_csv(input_path, output_path)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)